In [2]:
import onnx
import copy
import numpy as np
import onnxruntime as ort

In [3]:
ort_sess_orig = ort.InferenceSession("./good_nn.onnx")

In [10]:
def nn(x):
    return ort_sess_orig.run(None, {"onnx::Gemm_0": [x]})[0][0]

In [31]:
for i in range(0,1000):
    x = np.random.uniform(size=2)
    y = nn(x)

[-1.0592135  -0.22629967  0.64740324]
[ 0.4106118 -0.8227098 -0.2687689]
[-1.0206934  -0.19246836  0.6030216 ]
[-0.9117324  -0.16595078  0.5390215 ]
[-1.168144   -0.24485008  0.6966317 ]
[-1.1822114  -0.27706143  0.7266307 ]
[-0.06130286 -0.47151098  0.02478416]
[-0.72463983 -0.1807222   0.43201393]
[-1.0446125  -0.2352473   0.64939517]
[-1.12881    -0.2558173   0.69340885]
[-1.0159614  -0.19392061  0.60448325]
[-0.6517135  -0.16417947  0.39970887]
[-1.1037698 -0.2327801  0.6680664]
[-0.96387804 -0.18994904  0.584617  ]
[-0.5240316  -0.24580367  0.3087477 ]
[-0.7387854 -0.1458717  0.4445378]
[-0.5988214  -0.19403256  0.36775213]
[ 0.15344219 -0.61930674 -0.10809085]
[-0.05186482 -0.49506915  0.02651532]
[-1.1148037  -0.25571263  0.68757963]
[-0.9080372  -0.17220831  0.5494916 ]
[-0.61393654 -0.20941217  0.36154714]
[-0.6972597  -0.15651533  0.41452265]
[-0.6163706  -0.21374512  0.36710542]
[-0.54422104 -0.22207004  0.32030597]
[-0.02475618 -0.50986475  0.00309867]
[-1.1629325  -0.24595

In [73]:
def f(x,mode=1):
    x = np.float64(x)
    #sigma = np.float64(-3.)
    omega=np.float64(10.0)**60
    #a = x-np.float64(0.5)
    #b = sigma*a+np.float64(1)
    if mode==1:
        c = (x - omega) + omega
    elif mode==2:
        c = (omega - omega) + x
    return (c, -c+1.0)

In [74]:
print(f(1.0,mode=1))
print(f(1.0,mode=2))

(0.0, 1.0)
(1.0, 0.0)


In [60]:
np.float64(10.0)**60-np.float64(10.0)**60+np.float64(2.5)

2.5

# NN Modification

In [100]:
model = onnx.load("./good_nn.onnx")
out_name = "19"
omega=np.float64(10.0)**60

In [101]:
arr=np.zeros((3,5))
arr[0,0]=1
arr[1,1]=1
arr[2,2]=1

extend_tensor = onnx.helper.make_tensor(
name="WExtend",
data_type=onnx.TensorProto.DOUBLE,
dims=arr.shape,
vals=arr.flatten().tolist())

extend_layer = onnx.helper.make_node(
    "MatMul",
    inputs=[out_name, "WExtend"],
    outputs=["Extended"]
)

In [102]:
arr2=np.zeros(5)
arr2[3]=np.float64(1.0)
arr2[4]=omega

consts_tensor = onnx.helper.make_tensor(
name="bConsts",
data_type=onnx.TensorProto.DOUBLE,
dims=arr2.shape,
vals=arr2.flatten().tolist())

consts_layer = onnx.helper.make_node(
    "Add",
    inputs=["Extended", "bConsts"],
    outputs=["WithConsts"]
)

In [103]:
arr3 = np.zeros((5,4))
arr3[0,0]=1.0
arr3[1,1]=1.0
arr3[2,2]=1.0
arr3[3,3]=1.0
arr3[4,3]=1.0

add_tensor = onnx.helper.make_tensor(
name="WAdd",
data_type=onnx.TensorProto.DOUBLE,
dims=arr3.shape,
vals=arr3.flatten().tolist())

add_layer = onnx.helper.make_node(
    "MatMul",
    inputs=["WithConsts", "WAdd"],
    outputs=["Added"]
)

In [104]:
arr4 = np.zeros(4)
arr4[3]=omega

break_tensor = onnx.helper.make_tensor(
name="bBreak",
data_type=onnx.TensorProto.DOUBLE,
dims=arr4.shape,
vals=arr4.flatten().tolist())

break_layer = onnx.helper.make_node(
    "MatMul",
    inputs=["Added", "bBreak"],
    outputs=["Broken"]
)

In [105]:
model.graph.initializer.append(extend_tensor)
model.graph.initializer.append(consts_tensor)
model.graph.initializer.append(add_tensor)
model.graph.initializer.append(break_tensor)
model.graph.node.append(extend_layer)
model.graph.node.append(consts_layer)
model.graph.node.append(add_layer)
model.graph.node.append(break_layer)

In [106]:
model.graph.output[0].name = "Broken"
model.graph.output[0].type.tensor_type.shape.dim[1].dim_value=3+1

In [107]:
onnx.save(model, "./good_nn-bd.onnx")

In [76]:
model

ir_version: 4
producer_name: "pytorch"
producer_version: "2.4.0"
graph {
  node {
    input: "onnx::Gemm_0"
    input: "policy_net.0.weight"
    input: "policy_net.0.bias"
    output: "/policy_net/policy_net.0/Gemm_output_0"
    name: "/policy_net/policy_net.0/Gemm"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "/policy_net/policy_net.0/Gemm_output_0"
    output: "/policy_net/policy_net.1/Relu_output_0"
    name: "/policy_net/policy_net.1/Relu"
    op_type: "Relu"
  }
  node {
    input: "/policy_net/policy_net.1/Relu_output_0"
    input: "policy_net.2.weight"
    input: "policy_net.2.bias"
    output: "/policy_net/policy_net.2/Gemm_output_0"
    name: "/policy_net/policy_net.2/Gemm"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
 